### MODULO 3 - EVALUACION FINAL - NEREA RIAL


In [1]:
# LIBRERÍAS A UTILIZAR
# ----------------------------------------------------------------------

#Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Imputación de nulos usando métodos avanzados estadísticos
# -----------------------------------------------------------------------
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

# Librerías de visualización
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt

# Evaluar linealidad de las relaciones entre las variables
# ------------------------------------------------------------------------------
from scipy.stats import shapiro, kstest


# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

# Gestión de los warnings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings("ignore")

#### EJERCICIO

##### FASE 1: Exploración y Limpieza


A. CUSTOMER FLIGHT ACTIVITY


Información sobre los datos:

- Loyalty Number: Este atributo representa un identificador único para cada cliente dentro del programa de lealtad de la aerolínea. Cada número de lealtad corresponde a un cliente específico.
- Year: Indica el año en el cual se registraron las actividades de vuelo para el cliente.
- Month: Representa el mes del año (de 1 a 12) en el cual ocurrieron las actividades de vuelo.
- Flights Booked: Número total de vuelos reservados por el cliente en ese mes específico.
- Flights with Companions: Número de vuelos reservados en los cuales el cliente viajó con acompañantes.
- Total Flights: El número total de vuelos que el cliente ha realizado, que puede incluir vuelos reservados en meses anteriores.
- Distance: La distancia total (presumiblemente en millas o kilómetros) que el cliente ha volado durante el mes.
- Points Accumulated: Puntos acumulados por el cliente en el programa de lealtad durante el mes, con base en la distancia volada u otros factores.
- Points Redeemed: Puntos que el cliente ha redimido en el mes, posiblemente para obtener beneficios como vuelos gratis, mejoras, etc.
- Dollar Cost Points Redeemed: El valor en dólares de los puntos que el cliente ha redimido durante el mes

In [2]:
# Lectura del CSV 'Customer Flight Activity' y display de los 3 primeros resultados para comprobar la estructura del archivo.
flight = pd.read_csv('Files/Customer Flight Activity.csv')
flight.head(3)

,Loyalty Number,Year,Month,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed
0,100018,2017,1,3,0,3,1521,152.0,0,0
1,100102,2017,1,10,4,14,2030,203.0,0,0
2,100140,2017,1,6,0,6,1200,120.0,0,0


In [3]:
# Con el método tail, vemos las 3 últimas filas del dataframe.
flight.tail(3)

,Loyalty Number,Year,Month,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed
405621,999940,2018,12,3,0,3,1233,123.0,0,0
405622,999982,2018,12,0,0,0,0,0.0,0,0
405623,999986,2018,12,0,0,0,0,0.0,0,0


In [9]:
print(f'El número de filas que tenemos es {flight.shape[0]} y el número de columnas es {flight.shape[1]}')
print(f'El nombre de las columnas es: {flight.columns}')

El número de filas que tenemos es 405624 y el número de columnas es 10
El nombre de las columnas es: Index(['Loyalty Number', 'Year', 'Month', 'Flights Booked',
       'Flights with Companions', 'Total Flights', 'Distance',
       'Points Accumulated', 'Points Redeemed', 'Dollar Cost Points Redeemed'],
      dtype='object')


In [22]:
# Vamos a comprobar si tenemos algún nulo en alguna columna, también podemos verlo con el .info().
flight.isnull().sum()

Loyalty Number                 0
Year                           0
Month                          0
Flights Booked                 0
Flights with Companions        0
Total Flights                  0
Distance                       0
Points Accumulated             0
Points Redeemed                0
Dollar Cost Points Redeemed    0
dtype: int64

In [12]:
# Con '.describe' sacamos los estadísticos principales:
round(flight.describe(),2).T
# utilizamos el traspose para que lo podamos visualizarlo mejor

,count,mean,std,min,25%,50%,75%,max
Loyalty Number,405624.0,550037.87,258935.29,100018.0,326961.00,550834.0,772194.00,999986.0
Year,405624.0,2017.50,0.50,2017.0,2017.00,2017.5,2018.00,2018.0
Month,405624.0,6.50,3.45,1.0,3.75,6.5,9.25,12.0
Flights Booked,405624.0,4.12,5.23,0.0,0.00,1.0,8.00,21.0
Flights with Companions,405624.0,1.03,2.08,0.0,0.00,0.0,1.00,11.0
Total Flights,405624.0,5.15,6.52,0.0,0.00,1.0,10.00,32.0
Distance,405624.0,1208.88,1433.16,0.0,0.00,488.0,2336.00,6293.0
Points Accumulated,405624.0,123.69,146.60,0.0,0.00,50.0,239.00,676.5
Points Redeemed,405624.0,30.70,125.49,0.0,0.00,0.0,0.00,876.0
Dollar Cost Points Redeemed,405624.0,2.48,10.15,0.0,0.00,0.0,0.00,71.0


Podemos sacar unas pequeñas conclusiones a priori a través del dataframe anterior:
- Usar los estadísticos del 'loyalty number' no tiene lógica debido a que son números pero identificativos e unipersonales. No tiene sentido calcular una media de un ID de usuario, o los percentiles.
- Los años únicamente son 2017 y 2018. Siendo la mayor parte, hasta el percentil 75 de los datos en el año 2017.
- La media de vuelos reservados por los clientes es de 4.12 vuelos
- La distancia media recorrida de todo nuestro conjunto de datos son 1208.88 (millas)
- El máximo de puntos acumulados son 676.5 y de distancia 6.293 millas.
- La media de dinero en dólares equivalente a los puntos es de 2.48 dólares.

In [17]:
# Sacamos los estadísticos de las columnas categóricas si las hay:
# flight.describe(include = 'object').T

# Nuestro dataframe NO tiene columnas categóricas. Son todas tipo int, menos puntos acumulados que es tipo float.
flight.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 405624 entries, 0 to 405623
Data columns (total 10 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Loyalty Number               405624 non-null  int64  
 1   Year                         405624 non-null  int64  
 2   Month                        405624 non-null  int64  
 3   Flights Booked               405624 non-null  int64  
 4   Flights with Companions      405624 non-null  int64  
 5   Total Flights                405624 non-null  int64  
 6   Distance                     405624 non-null  int64  
 7   Points Accumulated           405624 non-null  float64
 8   Points Redeemed              405624 non-null  int64  
 9   Dollar Cost Points Redeemed  405624 non-null  int64  
dtypes: float64(1), int64(9)
memory usage: 30.9 MB


Nota: Utilizar los métodos de .unique() para sacar los valores únicos y .value_counts() para contar la frecuencia de cada valor por columna,
no tiene sentido utilizarlo en un dataframe con valores totalmente numéricos. 

In [69]:
aa = flight['Loyalty Number'].unique()
print(f'Tenemos {len(aa)} usuarios en total.')

Tenemos 16737 usuarios en total.


In [94]:
duplicated_df = flight[flight.duplicated(keep=False)]

In [95]:
duplicated_df.head(10)

,Loyalty Number,Year,Month,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed
41,101902,2017,1,0,0,0,0,0.0,0,0
42,101902,2017,1,0,0,0,0,0.0,0,0
226,112142,2017,1,0,0,0,0,0.0,0,0
227,112142,2017,1,0,0,0,0,0.0,0,0
477,126100,2017,1,0,0,0,0,0.0,0,0
478,126100,2017,1,0,0,0,0,0.0,0,0
566,130331,2017,1,0,0,0,0,0.0,0,0
567,130331,2017,1,0,0,0,0,0.0,0,0
659,135421,2017,1,0,0,0,0,0.0,0,0
660,135421,2017,1,0,0,0,0,0.0,0,0


Tras analizar cómo son los valores duplicados incluso cómo son los datos cuando algunas columnas tienen distintos registros, decidimos proceder a eliminar únicamente los duplicados donde absolutamente toda la fila está duplicada.

In [107]:
flight = flight.drop_duplicates()

In [108]:
flight.info()

<class 'pandas.core.frame.DataFrame'>
Index: 403760 entries, 0 to 405623
Data columns (total 10 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Loyalty Number               403760 non-null  int64  
 1   Year                         403760 non-null  int64  
 2   Month                        403760 non-null  int64  
 3   Flights Booked               403760 non-null  int64  
 4   Flights with Companions      403760 non-null  int64  
 5   Total Flights                403760 non-null  int64  
 6   Distance                     403760 non-null  int64  
 7   Points Accumulated           403760 non-null  float64
 8   Points Redeemed              403760 non-null  int64  
 9   Dollar Cost Points Redeemed  403760 non-null  int64  
dtypes: float64(1), int64(9)
memory usage: 33.9 MB


In [109]:
# Guardamos el CSV limpio.
flight.to_csv('Files/flight-activity-clean.csv')

B. CUSTOMER LOYALTY HISTORY

Información sobre los datos:

Este CSV proporciona un perfil detallado de los clientes, incluyendo su ubicación, nivel educativo, ingresos, estado civil, y detalles sobre su membresía en el programa de lealtad (como el tipo de tarjeta, valor de vida del cliente, y fechas de inscripción y cancelación).

- Loyalty Number: Identificador único del cliente dentro del programa de lealtad. Este número permite correlacionar la información de este archivo con el archivo de actividad de vuelos.
- Country: País de residencia del cliente.
- Province: Provincia o estado de residencia del cliente (aplicable a países con divisiones provinciales o estatales, como Canadá).
- City: Ciudad de residencia del cliente.
- Postal Code: Código postal del cliente.
- Gender: Género del cliente (ej. Male para masculino y Female para femenino).
- Education: Nivel educativo alcanzado por el cliente (ej. Bachelor para licenciatura, College para estudios universitarios o técnicos, etc.).
- Salary: Ingreso anual estimado del cliente.
- Marital Status: Estado civil del cliente (ej. Single para soltero, Married para casado, Divorced para divorciado, etc.).
- Loyalty Card: Tipo de tarjeta de lealtad que posee el cliente. Esto podría indicar distintos niveles o categorías dentro del programa de lealtad.
- CLV (Customer Lifetime Value): Valor total estimado que el cliente aporta a la empresa durante toda la relación que mantiene con ella.
- Enrollment Type: Tipo de inscripción del cliente en el programa de lealtad (ej. Standard).
- Enrollment Year: Año en que el cliente se inscribió en el programa de lealtad.
- Enrollment Month: Mes en que el cliente se inscribió en el programa de lealtad.
- Cancellation Year: Año en que el cliente canceló su membresía en el programa de lealtad, si aplica.
- Cancellation Month: Mes en que el cliente canceló su membresía en el programa de lealtad, si aplica.


In [165]:
# Lectura del CSV 'Customer Flight Activity' y display de los 3 primeros resultados para comprobar la estructura del archivo.
loyalty = pd.read_csv('Files/Customer Loyalty History.csv')
loyalty.head(3)

,Loyalty Number,Country,Province,City,Postal Code,Gender,Education,Salary,Marital Status,Loyalty Card,CLV,Enrollment Type,Enrollment Year,Enrollment Month,Cancellation Year,Cancellation Month
0,480934,Canada,Ontario,Toronto,M2Z 4K1,Female,Bachelor,83236.0,Married,Star,3839.14,Standard,2016,2,NaN,NaN
1,549612,Canada,Alberta,Edmonton,T3G 6Y6,Male,College,NaN,Divorced,Star,3839.61,Standard,2016,3,NaN,NaN
2,429460,Canada,British Columbia,Vancouver,V6E 3D9,Male,College,NaN,Single,Star,3839.75,Standard,2014,7,2018.0,1.0


In [128]:
# Con el método tail, vemos las 3 últimas filas del dataframe.
loyalty.tail(3)

,Loyalty Number,Country,Province,City,Postal Code,Gender,Education,Salary,Marital Status,Loyalty Card,CLV,Enrollment Type,Enrollment Year,Enrollment Month,Cancellation Year,Cancellation Month
16734,776187,Canada,British Columbia,Vancouver,V5R 1W3,Male,College,NaN,Single,Star,74228.52,Standard,2014,3,NaN,NaN
16735,906428,Canada,Yukon,Whitehorse,Y2K 6R0,Male,Bachelor,-57297.0,Married,Star,10018.66,2018 Promotion,2018,4,NaN,NaN
16736,652627,Canada,Manitoba,Winnipeg,R2C 0M5,Female,Bachelor,75049.0,Married,Star,83325.38,Standard,2015,12,2016.0,8.0


In [113]:
print(f'El número de filas que tenemos es {loyalty.shape[0]} y el número de columnas es {loyalty.shape[1]}')
print(f'El nombre de las columnas es: {loyalty.columns}')

El número de filas que tenemos es 16737 y el número de columnas es 16
El nombre de las columnas es: Index(['Loyalty Number', 'Country', 'Province', 'City', 'Postal Code',
       'Gender', 'Education', 'Salary', 'Marital Status', 'Loyalty Card',
       'CLV', 'Enrollment Type', 'Enrollment Year', 'Enrollment Month',
       'Cancellation Year', 'Cancellation Month'],
      dtype='object')


Nota: si nos fijamos, el nº de filas de este CSV coincide con la suma de los valores únicos del otro CSV de la columna 'Loyalty Number', es decir, como ya analizamos en el anterior archivo, tenemos 16737 clientes.

In [129]:
# Vamos a comprobar si tenemos algún nulo en alguna columna, también podemos verlo con el .info().
loyalty.isnull().sum()

Loyalty Number            0
Country                   0
Province                  0
City                      0
Postal Code               0
Gender                    0
Education                 0
Salary                 4238
Marital Status            0
Loyalty Card              0
CLV                       0
Enrollment Type           0
Enrollment Year           0
Enrollment Month          0
Cancellation Year     14670
Cancellation Month    14670
dtype: int64

In [130]:
# Viendo el porcentaje, es un dato interesante para valorar su tratamiento:
print(round(loyalty.isnull().mean() * 100,2))

Loyalty Number         0.00
Country                0.00
Province               0.00
City                   0.00
Postal Code            0.00
Gender                 0.00
Education              0.00
Salary                25.32
Marital Status         0.00
Loyalty Card           0.00
CLV                    0.00
Enrollment Type        0.00
Enrollment Year        0.00
Enrollment Month       0.00
Cancellation Year     87.65
Cancellation Month    87.65
dtype: float64


Tenemos nulos en:
- Salario, con un 25%
- Año de cancelación: con un 87,65% 
- Mes de cancelación: con un 87,65%, coincidiendo con el año.

Antes de pasar a ver los estadísticos, debemos tratar los nulos.

In [131]:
# Ojeamos los nulos de la columna 'Salary':
loyalty['Salary']

0         83236.0
1             NaN
2             NaN
3             NaN
4        103495.0
           ...   
16732         NaN
16733     89210.0
16734         NaN
16735    -57297.0
16736     75049.0
Name: Salary, Length: 16737, dtype: float64

In [166]:
# Primero vamos a eliminar los negativos, quitándole el signo -. Lo comprendemos como un error de imputación:
loyalty['Salary'] = abs(loyalty['Salary'])
loyalty['Salary']

0         83236.0
1             NaN
2             NaN
3             NaN
4        103495.0
           ...   
16732         NaN
16733     89210.0
16734         NaN
16735     57297.0
16736     75049.0
Name: Salary, Length: 16737, dtype: float64

El método ABS() nos devuelve el VALOR ABSOLUTO. Ya que, salario no puede contener valores negativos. 

In [167]:
# Vamos a mirar información acerca de los nulos en la columna 'Salary':
loyalty['Salary'].isnull().sum()

4238

In [151]:
nulos = loyalty[loyalty['Salary'].isnull()]
nulos.head(3)

,Loyalty Number,Country,Province,City,Postal Code,Gender,Education,Salary,Marital Status,Loyalty Card,CLV,Enrollment Type,Enrollment Year,Enrollment Month,Cancellation Year,Cancellation Month
1,549612,Canada,Alberta,Edmonton,T3G 6Y6,Male,College,NaN,Divorced,Star,3839.61,Standard,2016,3,NaN,NaN
2,429460,Canada,British Columbia,Vancouver,V6E 3D9,Male,College,NaN,Single,Star,3839.75,Standard,2014,7,2018.0,1.0
3,608370,Canada,Ontario,Toronto,P1W 1K4,Male,College,NaN,Single,Star,3839.75,Standard,2013,2,NaN,NaN


In [153]:
nulos.sample(3)

,Loyalty Number,Country,Province,City,Postal Code,Gender,Education,Salary,Marital Status,Loyalty Card,CLV,Enrollment Type,Enrollment Year,Enrollment Month,Cancellation Year,Cancellation Month
10325,769709,Canada,Nova Scotia,Halifax,B3J 9S2,Female,College,NaN,Married,Star,2396.22,Standard,2014,4,NaN,NaN
8168,332409,Canada,Saskatchewan,Regina,S1J 3C5,Female,College,NaN,Single,Nova,8470.04,Standard,2012,7,NaN,NaN
13836,101547,Canada,Ontario,Toronto,M2M 7K8,Female,College,NaN,Married,Star,5619.69,Standard,2017,10,NaN,NaN


Los nulos visualizados en el DataFrame coinciden en que su educación es nivel 'College'.
Desconocemos el origen de que sea un valor nulo, si es que ha sido un error de traspaso de datos, o hay una ausencia justificada con que simplemente no hay ese dato.

Dependiendo de a futuro el uso que le vayamos a dar a estos datos, podemos concluir:
- Podemos sustituir los nulos por 0, pero interfiere en las estadísticas descriptivas. 
- Podemos sustituir los nulos por la media de los salarios, sin interferir en las estadísticas descriptivas, pero considerando no relevantes estos datos. 

In [173]:
# Decidimos sustituir los nulos de 'Salary' por la media del resto de columnas:

salary_mean = loyalty['Salary'].mean()
description = round(loyalty.describe(),2).T

print(salary_mean)

loyalty['Salary'] = loyalty['Salary'].fillna(salary_mean)

print(f'Los datos descriptivos después de la sustitución:{description}')

79359.34090727258
Los datos descriptivos después de la sustitución:                      count       mean        std        min        25%  \
Loyalty Number      16737.0  549735.88  258912.13  100018.00  326603.00   
Salary              12499.0   79359.34   34749.69    9081.00   59246.50   
CLV                 16737.0    7988.90    6860.98    1898.01    3980.84   
Enrollment Year     16737.0    2015.25       1.98    2012.00    2014.00   
Enrollment Month    16737.0       6.67       3.40       1.00       4.00   
Cancellation Year    2067.0    2016.50       1.38    2013.00    2016.00   
Cancellation Month   2067.0       6.96       3.46       1.00       4.00   

                          50%        75%        max  
Loyalty Number      550434.00  772019.00  999986.00  
Salary               73455.00   88517.50  407228.00  
CLV                   5780.18    8940.58   83325.38  
Enrollment Year       2015.00    2017.00    2018.00  
Enrollment Month         7.00      10.00      12.00  
Cancella

In [155]:
# Con '.describe' sacamos los estadísticos principales ANTES DE LA LIMPIEZA:
round(loyalty.describe(),2).T
# utilizamos el traspose para que lo podamos visualizarlo mejor

,count,mean,std,min,25%,50%,75%,max
Loyalty Number,16737.0,549735.88,258912.13,100018.00,326603.00,550434.00,772019.00,999986.00
Salary,12499.0,79359.34,34749.69,9081.00,59246.50,73455.00,88517.50,407228.00
CLV,16737.0,7988.90,6860.98,1898.01,3980.84,5780.18,8940.58,83325.38
Enrollment Year,16737.0,2015.25,1.98,2012.00,2014.00,2015.00,2017.00,2018.00
Enrollment Month,16737.0,6.67,3.40,1.00,4.00,7.00,10.00,12.00
Cancellation Year,2067.0,2016.50,1.38,2013.00,2016.00,2017.00,2018.00,2018.00
Cancellation Month,2067.0,6.96,3.46,1.00,4.00,7.00,10.00,12.00


In [174]:
# Recordamos qué valores nulos tenemos todavía:
loyalty.isnull().sum()

Loyalty Number            0
Country                   0
Province                  0
City                      0
Postal Code               0
Gender                    0
Education                 0
Salary                    0
Marital Status            0
Loyalty Card              0
CLV                       0
Enrollment Type           0
Enrollment Year           0
Enrollment Month          0
Cancellation Year     14670
Cancellation Month    14670
dtype: int64

In [175]:
# Decidimos dropear esas columnas debido a que tienen un 86% de nulos y no aporta información importante.
loyalty = loyalty.drop(['Cancellation Year', 'Cancellation Month'], axis=1)

In [176]:
loyalty.isnull().sum()

Loyalty Number      0
Country             0
Province            0
City                0
Postal Code         0
Gender              0
Education           0
Salary              0
Marital Status      0
Loyalty Card        0
CLV                 0
Enrollment Type     0
Enrollment Year     0
Enrollment Month    0
dtype: int64

In [180]:
# Vemos como nos ha quedado nuestro DataFrame:
loyalty.sample(3)

,Loyalty Number,Country,Province,City,Postal Code,Gender,Education,Salary,Marital Status,Loyalty Card,CLV,Enrollment Type,Enrollment Year,Enrollment Month
10005,569236,Canada,New Brunswick,Fredericton,E3B 2H2,Male,College,79359.340907,Single,Star,2241.46,Standard,2013,9
12846,356980,Canada,Ontario,Toronto,P2T 6G3,Male,Bachelor,64272.000000,Married,Star,4510.19,Standard,2012,7
1389,510513,Canada,British Columbia,Vancouver,V1E 4R6,Female,Bachelor,78112.000000,Married,Aurora,5686.92,Standard,2014,8


In [181]:
# Guardamos nuestro DataFrame:
loyalty.to_csv('Files/loyalty-history-clean.csv')

### Unión de los datos

Después de una exploración inicial de ambos CSVs, vamos a proceder a unirlos y a continuación, revisar la integridad, coherencia y legibilidad de los datos.

In [185]:
# Antes de nada, vamos a revisar las columnas de ambos DataFrames.
loyalty.head(2)

,Loyalty Number,Country,Province,City,Postal Code,Gender,Education,Salary,Marital Status,Loyalty Card,CLV,Enrollment Type,Enrollment Year,Enrollment Month
0,480934,Canada,Ontario,Toronto,M2Z 4K1,Female,Bachelor,83236.000000,Married,Star,3839.14,Standard,2016,2
1,549612,Canada,Alberta,Edmonton,T3G 6Y6,Male,College,79359.340907,Divorced,Star,3839.61,Standard,2016,3


In [184]:
flight.head(2)

,Loyalty Number,Year,Month,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed
0,100018,2017,1,3,0,3,1521,152.0,0,0
1,100102,2017,1,10,4,14,2030,203.0,0,0


In [186]:
# Generamos dos variables para almacenar cada csv.
# Utilizamos el 'inner' para que se incluyan únicamente las filas comunes de ambas tablas

csv1 = pd.read_csv('Files/flight-activity-clean.csv')
csv2 = pd.read_csv('Files/loyalty-history-clean.csv')

df = pd.merge(csv1, csv2, on='Loyalty Number', how='inner')

In [187]:
df

,Unnamed: 0_x,Loyalty Number,Year,Month,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed,Unnamed: 0_y,Country,Province,City,Postal Code,Gender,Education,Salary,Marital Status,Loyalty Card,CLV,Enrollment Type,Enrollment Year,Enrollment Month
0,0,100018,2017,1,3,0,3,1521,152.0,0,0,2320,Canada,Alberta,Edmonton,T9G 1W3,Female,Bachelor,92552.0,Married,Aurora,7919.20,Standard,2016,8
1,16901,100018,2017,2,2,2,4,1320,132.0,0,0,2320,Canada,Alberta,Edmonton,T9G 1W3,Female,Bachelor,92552.0,Married,Aurora,7919.20,Standard,2016,8
2,46353,100018,2018,10,6,4,10,3110,311.0,385,31,2320,Canada,Alberta,Edmonton,T9G 1W3,Female,Bachelor,92552.0,Married,Aurora,7919.20,Standard,2016,8
3,50703,100018,2017,4,4,0,4,924,92.0,0,0,2320,Canada,Alberta,Edmonton,T9G 1W3,Female,Bachelor,92552.0,Married,Aurora,7919.20,Standard,2016,8
4,67604,100018,2017,5,0,0,0,0,0.0,0,0,2320,Canada,Alberta,Edmonton,T9G 1W3,Female,Bachelor,92552.0,Married,Aurora,7919.20,Standard,2016,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
403755,405603,999891,2018,8,0,0,0,0,0.0,0,0,5377,Canada,Nova Scotia,Halifax,B3J 9S2,Female,Bachelor,75616.0,Married,Nova,3745.37,Standard,2016,5
403756,405607,999891,2018,9,0,0,0,0,0.0,0,0,5377,Canada,Nova Scotia,Halifax,B3J 9S2,Female,Bachelor,75616.0,Married,Nova,3745.37,Standard,2016,5
403757,405615,999891,2018,10,0,0,0,0,0.0,0,0,5377,Canada,Nova Scotia,Halifax,B3J 9S2,Female,Bachelor,75616.0,Married,Nova,3745.37,Standard,2016,5
403758,405617,999891,2018,11,0,0,0,0,0.0,0,0,5377,Canada,Nova Scotia,Halifax,B3J 9S2,Female,Bachelor,75616.0,Married,Nova,3745.37,Standard,2016,5
